In [1]:
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import hub
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchaudio

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, recall_score, precision_score, balanced_accuracy_score, accuracy_score, classification_report
from sklearn.utils import shuffle

import scipy

from tqdm import tqdm

from datasets import load_dataset, Dataset, Audio
import librosa
from transformers import AutoFeatureExtractor, Wav2Vec2ForSequenceClassification

from models.basic_transformer import BasicTransformer

from src.utils import AphasiaDatasetMFCC, AphasiaDatasetSpectrogram, AphasiaDatasetWaveform

from collections import Counter
from models.wav2vecClassifier import Wav2vecClassifier
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Gain

2025-04-15 22:41:12.859446: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 22:41:12.975621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744746073.040935   12758 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744746073.055814   12758 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 22:41:13.167011: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
AUDIO_LENGTH = 6_000
SEQUENCE_LENGTH = 31
MFCC = 128
print(f"It's {DEVICE} time!!!")

It's cuda time!!!


In [3]:
DATA_DIR = os.path.join('..', 'data')
VOICES_DIR = os.path.join(DATA_DIR, 'Voices_wav')
APHASIA_DIR = os.path.join(VOICES_DIR, 'Aphasia')
NORM_DIR = os.path.join(VOICES_DIR, 'Norm')
RIR_DIR = os.path.join(DATA_DIR, 'RIRs')
NOISE_DIR = os.path.join(DATA_DIR, 'noise')

In [4]:
augmentations = audio_augment = Compose([
                AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.02, p=0.7),
                TimeStretch(min_rate=0.4, max_rate=1.5, p=0.5),
                PitchShift(min_semitones=-8, max_semitones=8, p=0.5),
                Shift(min_shift=-0.8, max_shift=0.8, p=0.5),
                Gain(min_gain_db=-14, max_gain_db=14, p=0.5),
            ])

In [5]:
train_dataset = AphasiaDatasetSpectrogram(os.path.join(DATA_DIR, "train_filenames_mc_1.csv"), VOICES_DIR, snr=(10, 20), rirs_dir=RIR_DIR, target_sample_rate=8_000, file_format="wav", transforms=augmentations)

KeyboardInterrupt: 

In [ ]:
test_dataset = AphasiaDatasetSpectrogram(os.path.join(DATA_DIR, "val_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, file_format="wav")

In [ ]:
val_dataset = AphasiaDatasetSpectrogram(os.path.join(DATA_DIR, "test_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, file_format="wav")

In [ ]:
# Балансировка классов для train
train_labels = [label for _, label in train_dataset.data]
class_counts = Counter(train_labels)
if len(class_counts) < 4:
    raise ValueError("Один из классов отсутствует в тренировочном наборе")

class_weights = {label: 1.0 / count for label, count in class_counts.items()}
weights = [class_weights[label] for _, label in train_dataset.data]
train_sampler = WeightedRandomSampler(weights, num_samples=len(train_dataset), replacement=True)

In [ ]:
MAX_LEN = 120_000

In [ ]:
def pad_sequence(batch):
    if not batch:
        return torch.zeros(0), torch.zeros(0)
    
    spectrograms, labels = zip(*batch)
    max_len = max(s.shape[1] for s in spectrograms)
    freq_bins = spectrograms[0].shape[2]
    
    padded = torch.zeros(len(spectrograms), 1, max_len, freq_bins)
    for i, s in enumerate(spectrograms):
        padded[i, :, :s.shape[1], :] = s
    
    return padded, torch.stack(labels) 

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, collate_fn=pad_sequence, drop_last=True, num_workers=6)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=pad_sequence, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=pad_sequence, drop_last=True, num_workers=6)

In [ ]:
def train_model(model, dl_train, dl_val, epochs=1, lr=0.001, device="cpu"):
      
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights.values()))).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-3)
        
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-7, last_epoch=-1) # torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3, threshold=1e-3)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3, threshold=1e-3)
        
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = []
    for epoch in tqdm(range(epochs), desc="Training model"):
        model.train()
        total_train_loss = 0
        total_val_loss = 0
        
        train_acc = []
        val_acc = []
        for features, target in dl_train:
            features, target = features.to(device), target.to(device)
            optimizer.zero_grad()
            
            output = model(features).squeeze()

            preds = torch.argmax(output, dim=1).cpu().detach().numpy()
            train_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.detach().item()
        
        avg_train_acc = np.stack(train_acc, axis=0).mean()

        avg_train_loss = total_train_loss / len(train_dataloader)
        train_loss_list.append(avg_train_loss)
        train_acc_list.append(avg_train_acc)
                
        model.eval()
        
        with torch.no_grad():
            for features, target in dl_val:
                features, target = features.to(device), target.to(device)
                
                output = model(features).squeeze()
                
                preds = torch.argmax(output, dim=1).cpu().detach().numpy()
                val_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
                loss = criterion(output, target)
                total_val_loss += loss.detach().item()
        
        avg_val_acc = np.stack(val_acc, axis=0).mean()
        avg_val_loss = total_val_loss / len(dl_val)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        
        if scheduler:
            try:
                scheduler.step()
            except:
                scheduler.step(avg_val_loss)
        
        # if epoch % 10 == 0:
        tqdm.write(f"Epoch {epoch}: train loss: {avg_train_loss:.3f}, train balanced acc: {avg_train_acc:.2f}, test loss: {avg_val_loss:.3f}, test balanced acc: {avg_val_acc:.2f}, lr: {optimizer.param_groups[0]['lr']}")

            
    return model, train_loss_list, val_loss_list, train_acc_list, val_acc_list

In [13]:
from models.cnn import MobileNet

cnn = MobileNet(num_classes=4)

In [14]:
cnn, train_l, val_l, train_accuracy, val_accuracy = train_model(cnn, train_dataloader, val_dataloader, epochs=130, lr=1e-3, device=DEVICE)

Training model:   1%|          | 1/130 [00:13<28:26, 13.23s/it]

Epoch 0: train loss: 1.360, train balanced acc: 0.26, test loss: 1.346, test balanced acc: 0.30, lr: 0.001


Training model:   2%|▏         | 2/130 [00:26<27:55, 13.09s/it]

Epoch 1: train loss: 1.314, train balanced acc: 0.28, test loss: 1.346, test balanced acc: 0.30, lr: 0.001


Training model:   2%|▏         | 3/130 [00:39<27:36, 13.04s/it]

...

Epoch 51: train loss: 0.293, train balanced acc: 0.86, test loss: 2.793, test balanced acc: 0.37, lr: 4.8828125e-07


Training model:  41%|████      | 53/130 [11:40<17:09, 13.37s/it]

Epoch 52: train loss: 0.307, train balanced acc: 0.85, test loss: 2.802, test balanced acc: 0.38, lr: 2.44140625e-07


Training model:  41%|████      | 53/130 [11:48<17:09, 13.37s/it]


KeyboardInterrupt: 